In [243]:
import sympy as sp
import numpy as np
from sympy import pi
from sympy.utilities import lambdify
from scipy.integrate import quad, quadrature
from scipy.optimize import root_scalar, fsolve

from parameters import *
from functions import *

mpi = sp.S.One
one = sp.S.One
a, mu, p = sp.symbols("a, mu, p", real=True)


In [284]:
# Mass-parameters
m1_sq = mpi**2 * sp.cos(a) - mu**2 * sp.cos(2 * a)
m2_sq = mpi**2 * sp.cos(a) - mu**2 * sp.cos(a)**2
m3_sq = mpi**2 * sp.cos(a) + mu**2 * sp.sin(a)**2
m12 = 2 * mu * sp.cos(a)

# square of the dispertion relations of the particles
E0_sq = p**2 + m2_sq

M_sq = (m1_sq + m2_sq + m12**2)

Ep_sq = p**2 + 1 / 2 * M_sq + 1/2 * sp.sqrt(4 * p**2 * m12**2 + M_sq**2 - 4*m1_sq*m2_sq)
Em_sq = p**2 + 1 / 2 * M_sq - 1/2 * sp.sqrt(4 * p**2 * m12**2 + M_sq**2 - 4*m1_sq*m2_sq)

m0sq = m3_sq
mp_sq = 1 / 2 * M_sq + 1/2 * sp.sqrt(M_sq**2 - 4*m1_sq*m2_sq)
mm_sq = 1 / 2 * M_sq - 1/2 * sp.sqrt(M_sq**2 - 4*m1_sq*m2_sq)

# Masss/energies used for F_fin
mtilde1_sq = m3_sq
mtilde2_sq = mpi**2 * sp.cos(a)

E1_sq = p**2 + mtilde1_sq
E2_sq = p**2 + mtilde2_sq

F_0_2 = - f**2 * (mpi**2 * sp.cos(a) + 1/2*mu**2 * sp.sin(a)**2)


ls = [sp.symbols("l{}".format(i)) for i in range(0, 5)]
F_0_4 = \
    - one/2 * 1 / (4 * sp.pi)**2 * (
    one/3 * ( ls[1] + 2 * ls[2] + one*3/2 - 3*sp.log(m3_sq) ) * mu**4 * sp.sin(a)**4
    + one/2*(-ls[3] + 4*ls[4] + one*3/2 - 2*sp.log(m3_sq) - sp.log(mtilde2_sq) )*mpi**4*sp.cos(a)**2
    + 2 * (ls[4] - one/2 - sp.log(m3_sq)) * mpi**2 *mu**2 * sp.cos(a)*sp.sin(a)**2
)

F_0_4 = F_0_4.subs(ls[1],l1).subs(ls[2],l2).subs(ls[3], l3).subs(ls[4],l4)

# Integrand of F_fin
dF_fin = 4 / (4 * pi)**2 * p**2 * (sp.sqrt(Ep_sq) + sp.sqrt(Em_sq) - sp.sqrt(E1_sq) - sp.sqrt(E2_sq))
F_fin = lambda mu, alpha: quad(dF_fin_diff_a, 0.0001, 40, args=(mu, alpha))[0]

F1 = lambdify((mu, a), F_0_2)
F2 = lambdify((mu, a), F_0_4)
F = lambda mu, alpha: F1(mu, alpha) + F2(mu, alpha) + F_fin(mu, alpha)


In [285]:
# ls = [sp.symbols("l{}".format(i)) for i in range(0, 5)]
# F_0_4 = \
#     - one/2 * 1 / (4 * sp.pi)**2 * (
#     one/3 * ( ls[1] + 2 * ls[2] + one*3/2 - 3*sp.log(m3_sq) ) * mu**4 * sp.sin(a)**4
#     + one/2*(-ls[3] + 4*ls[4] + one*3/2 - 2*sp.log(m3_sq) - sp.log(mtilde2_sq) )*mpi**4*sp.cos(a)**2
#     + 2 * (ls[4] - one/2 - sp.log(m3_sq)) * mpi**2 *mu**2 * sp.cos(a)*sp.sin(a)**2
# )

F_0_4

-(mu**4*(3.23333333333333 - log(mu**2*sin(a)**2 + cos(a)))*sin(a)**4 + mu**2*(7.8 - 2*log(mu**2*sin(a)**2 + cos(a)))*sin(a)**2*cos(a) + (-log(mu**2*sin(a)**2 + cos(a)) - log(cos(a))/2 + 8.1)*cos(a)**2)/(32*pi**2)

In [286]:
F_0_2_diff_a = lambdify((mu, a), F_0_2.diff(a))
F_0_4_diff_a = lambdify((mu, a), F_0_2.diff(a))
dF_fin_diff_a = lambdify((p, mu, a), dF_fin.diff(a))
F_fin_diff_a = lambda mu, alpha: quad(dF_fin_diff_a, 0.0001, 10, args=(mu, alpha))[0]

F_diff_a = lambda mu, alpha: \
    F_0_2_diff_a(mu, alpha) + F_0_4_diff_a(mu, alpha) + F_fin_diff_a(mu, alpha)

In [287]:
m = 1.4
x0 = alpha_0(np.array([m]))[0]
g = lambda x: F_diff_a(m, x)

print(x0)
print(fsolve(g, x0=x0))


1.0353742637169008


<lambdifygenerated-99>:2: RuntimeWarning: invalid value encountered in sqrt
  return (0.0253302959105844*p**2*(-(mu**2*sin(a)*cos(a) - 1/2*sin(a))/sqrt(mu**2*sin(a)**2 + p**2 + cos(a)) + 1.22474487139159*(-mu**2*sin(a)*cos(a) + 0.333333333333333*mu**2*sin(2*a) - 0.333333333333333*sin(a) - 0.166666666666667*(-16*mu**2*p**2*sin(a)*cos(a) + (1/2)*(-8*mu**2*sin(2*a) + 4*sin(a))*(-mu**2*cos(a)**2 + cos(a)) + (1/2)*(4*mu**2*cos(2*a) - 4*cos(a))*(2*mu**2*sin(a)*cos(a) - sin(a)) + (1/2)*(3*mu**2*cos(a)**2 - mu**2*cos(2*a) + 2*cos(a))*(-12*mu**2*sin(a)*cos(a) + 4*mu**2*sin(2*a) - 4*sin(a)))/sqrt(16*mu**2*p**2*cos(a)**2 - (-mu**2*cos(a)**2 + cos(a))*(-4*mu**2*cos(2*a) + 4*cos(a)) + (3*mu**2*cos(a)**2 - mu**2*cos(2*a) + 2*cos(a))**2))/sqrt(mu**2*cos(a)**2 - 0.333333333333333*mu**2*cos(2*a) + 0.666666666666667*p**2 - 0.333333333333333*sqrt(16*mu**2*p**2*cos(a)**2 - (-mu**2*cos(a)**2 + cos(a))*(-4*mu**2*cos(2*a) + 4*cos(a)) + (3*mu**2*cos(a)**2 - mu**2*cos(2*a) + 2*cos(a))**2) + 0.666666666666667*c

[1.03537426]


/home/martin/.local/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [289]:
print(F(m, x0 + 0.01))
print(F(m, x0))
print(F(m, x0 - 0.0001))

-0.5482130247203346
-0.5481688332746166
nan


/tmp/ipykernel_37552/2588864700.py:41: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  F_fin = lambda mu, alpha: quad(dF_fin_diff_a, 0.0001, 40, args=(mu, alpha))[0]


In [280]:
mus = mu_list[1::]
alphas = alpha_0(mus)
d = 0.001
z = [(F(mu, a-d), F(mu, a), F(mu, a+d) )for mu, a in zip(mus, alphas)]

<lambdifygenerated-88>:2: RuntimeWarning: invalid value encountered in sqrt
  return (0.0253302959105844*p**2*(-(mu**2*sin(a)*cos(a) - 1/2*sin(a))/sqrt(mu**2*sin(a)**2 + p**2 + cos(a)) + 1.22474487139159*(-mu**2*sin(a)*cos(a) + 0.333333333333333*mu**2*sin(2*a) - 0.333333333333333*sin(a) - 0.166666666666667*(-16*mu**2*p**2*sin(a)*cos(a) + (1/2)*(-8*mu**2*sin(2*a) + 4*sin(a))*(-mu**2*cos(a)**2 + cos(a)) + (1/2)*(4*mu**2*cos(2*a) - 4*cos(a))*(2*mu**2*sin(a)*cos(a) - sin(a)) + (1/2)*(3*mu**2*cos(a)**2 - mu**2*cos(2*a) + 2*cos(a))*(-12*mu**2*sin(a)*cos(a) + 4*mu**2*sin(2*a) - 4*sin(a)))/sqrt(16*mu**2*p**2*cos(a)**2 - (-mu**2*cos(a)**2 + cos(a))*(-4*mu**2*cos(2*a) + 4*cos(a)) + (3*mu**2*cos(a)**2 - mu**2*cos(2*a) + 2*cos(a))**2))/sqrt(mu**2*cos(a)**2 - 0.333333333333333*mu**2*cos(2*a) + 0.666666666666667*p**2 - 0.333333333333333*sqrt(16*mu**2*p**2*cos(a)**2 - (-mu**2*cos(a)**2 + cos(a))*(-4*mu**2*cos(2*a) + 4*cos(a)) + (3*mu**2*cos(a)**2 - mu**2*cos(2*a) + 2*cos(a))**2) + 0.666666666666667*c

In [279]:
z

[(-0.4393297735494559, -0.4393297758491358, -0.4393297735494557),
 (-0.43932977355382524, -0.4393297758491358, -0.43932977355382413),
 (-0.4393297735611058, -0.4393297758491358, -0.4393297735611066),
 (-0.43932977357130104, -0.4393297758491358, -0.4393297735712997),
 (-0.43932977358440595, -0.4393297758491358, -0.4393297735844083),
 (-0.43932977360042613, -0.4393297758491358, -0.4393297736004268),
 (-0.43932977361935904, -0.4393297758491358, -0.43932977361935793),
 (-0.4393297736412054, -0.4393297758491358, -0.43932977364120085),
 (-0.4393297736659625, -0.4393297758491358, -0.43932977366595816),
 (-0.4393297736936325, -0.4393297758491358, -0.43932977369362775),
 (-0.4393297737242111, -0.4393297758491358, -0.4393297737242141),
 (-0.4393297737577068, -0.4393297758491358, -0.4393297737577084),
 (-0.43932977379411414, -0.4393297758491358, -0.43932977379411636),
 (-0.4393297738334281, -0.4393297758491358, -0.43932977383344307),
 (-0.4393297738756639, -0.4393297758491358, -0.439329773875673)

In [327]:
n = 15
x = np.linspace(0, 2.5, n)
y = np.linspace(0, pi/2, n)
xx, yy = np.meshgrid(x, y)

g = lambdify((p, mu, a), Em_sq)
arr = np.sqrt(g(0, xx, yy))
np.array(np.isnan(arr), dtype=int).T


/tmp/ipykernel_37552/3600362512.py:7: RuntimeWarning: invalid value encountered in sqrt
  arr = np.sqrt(g(0, xx, yy))


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]])